# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание проекта
<br>Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.</br>

Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

<br>сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
</br>

In [1]:
#!pip install -U scikit-learn
import pandas as pd
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.dummy import DummyClassifier


pd.set_option('display.float_format', '{:,.2f}'.format)

## Откройте и изучите файл

откроем файл и предобработаем данные

In [2]:
df = pd.read_csv(r"C:\Users\lebed\Downloads\users_behavior.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.00,311.90,83.00,"19,915.42",0
1,85.00,516.75,56.00,"22,696.96",0
2,77.00,467.66,86.00,"21,060.45",0
3,106.00,745.53,81.00,"8,437.39",1
4,66.00,418.74,1.00,"14,502.75",0
5,58.00,344.56,21.00,"15,823.37",0
6,57.00,431.64,20.00,"3,738.90",1
7,15.00,132.40,6.00,"21,911.60",0
8,7.00,43.39,3.00,"2,538.67",1
9,90.00,665.41,38.00,"17,358.61",0


In [4]:
df.tail(10)

,calls,minutes,messages,mb_used,is_ultra
3204,86.00,658.66,47.00,"14,153.10",0
3205,59.00,412.81,16.00,"14,105.03",0
3206,76.00,586.51,54.00,"14,345.74",0
3207,17.00,92.39,2.00,"4,299.25",0
3208,164.00,"1,016.98",71.00,"17,787.52",1
3209,122.00,910.98,20.00,"35,124.90",1
3210,25.00,190.36,0.00,"3,275.61",0
3211,97.00,634.44,70.00,"13,974.06",0
3212,64.00,462.32,90.00,"31,239.78",0
3213,80.00,566.09,6.00,"29,480.52",1


In [5]:
df["is_ultra"].value_counts()

is_ultra
0    2229
1     985
Name: count, dtype: int64

In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,"3,214.00","3,214.00","3,214.00","3,214.00","3,214.00"
mean,63.04,438.21,38.28,"17,207.67",0.31
std,33.24,234.57,36.15,"7,570.97",0.46
min,0.00,0.00,0.00,0.00,0.00
25%,40.00,274.58,9.00,"12,491.90",0.00
50%,62.00,430.60,30.00,"16,943.24",0.00
75%,82.00,571.93,57.00,"21,424.70",1.00
max,244.00,"1,632.06",224.00,"49,745.73",1.00


In [7]:
df.duplicated().sum()

0

In [8]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Открыли и изучили основную информацию по сету. Получили информацию в 3214 строк и 5 столбцов, с верным стилем в заголовках, верным форматом данных , без дубликатов и без пропусков. Также, посчитали кол-во клиентов по подписке ультра. 

Оценим взаимную корреляцию  данных

In [9]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.00,0.98,0.18,0.29,0.21
minutes,0.98,1.00,0.17,0.28,0.21
messages,0.18,0.17,1.00,0.20,0.20
mb_used,0.29,0.28,0.20,1.00,0.20
is_ultra,0.21,0.21,0.20,0.20,1.00


Из данных очевидно, что высокая корреляция (~1 ) между данными по столбцу calls и minutes,в остальных случаях корреляция довольно низкая.
Что бы избежать проблемы мультиколлениарности, будем использовать только один столбец calls, minutes удалять из таблицы не будем.


## Разбейте данные на выборки

 Разделим исходные данные на обучающую, валидационную и тестовую выборки.

In [10]:
features = df.drop(['minutes', 'is_ultra'], axis=1) 
target = df['is_ultra'] 

In [11]:
display(features.shape)
display(target.shape)

(3214, 3)

(3214,)

в связи с заданием, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1. 

In [12]:
features_base, features_test, target_base, target_test = train_test_split(
    features, target, test_size=0.20, random_state=12345)

In [13]:
display(features_base.shape)
display(target_base.shape)
display(features_test.shape)
display(target_test.shape)

(2571, 3)

(2571,)

(643, 3)

(643,)

In [14]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_base, target_base, test_size=0.25, random_state=12345)

In [15]:
display(features_train.shape)
display(features_valid.shape)
display(features_test.shape)

(1928, 3)

(643, 3)

(643, 3)

разбили данные на обучающу, валидиционную и тестовую в соотношении 3:1:1


## Исследуйте модели

Исследуем качество разных моделей, меняя гиперпараметры

Модель дерево решений


In [16]:
model = DecisionTreeClassifier(random_state=12345)

model.fit(features_train, target_train)

DecisionTreeClassifier(random_state=12345)

In [17]:
train_predictions = model.predict(features_train)

valid_predictions = model.predict(features_valid)
print("Accuracy")
print("Обучающая выборка:", accuracy_score(target_train, train_predictions))

print("Валидационная выборка:", accuracy_score(target_valid,valid_predictions))

Accuracy
Обучающая выборка: 1.0
Валидационная выборка: 0.7107309486780715


In [18]:
best_model = None
best_result = 0
for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predictions_valid = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions_valid) 
    if result > best_result:
        best_model = model 
        best_result = result 
    print("max_depth =", depth, ": ", end='')        
    print("Accuracy лучшей модели:", result)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)   

max_depth = 1 : Accuracy лучшей модели: 0.7387247278382582
max_depth = 2 : Accuracy лучшей модели: 0.7589424572317263
max_depth = 3 : Accuracy лучшей модели: 0.7651632970451011
max_depth = 4 : Accuracy лучшей модели: 0.7620528771384136
max_depth = 5 : Accuracy лучшей модели: 0.76049766718507
max_depth = 6 : Accuracy лучшей модели: 0.7573872472783826
max_depth = 7 : Accuracy лучшей модели: 0.7573872472783826
max_depth = 8 : Accuracy лучшей модели: 0.7682737169517885
max_depth = 9 : Accuracy лучшей модели: 0.7620528771384136
max_depth = 10 : Accuracy лучшей модели: 0.7729393468118196
max_depth = 11 : Accuracy лучшей модели: 0.7667185069984448
max_depth = 12 : Accuracy лучшей модели: 0.7573872472783826
max_depth = 13 : Accuracy лучшей модели: 0.7542768273716952
max_depth = 14 : Accuracy лучшей модели: 0.7356143079315708
Accuracy наилучшей модели на валидационной выборке: 0.7729393468118196


Наилучшая модель на валидационной выборке при использовании дерево решений при  accuracy = 0.773, глубина дерева depth = 10

Модель случайный лес

In [19]:
best_model_forest = None
best_result_forest = 0
best_est_forest = 0
best_depth_forest = 0

for est in range(1, 11):
    for depth in range (1,15):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth) 
        model_forest.fit(features_train, target_train) 
        result_forest = model_forest.score(features_valid, target_valid) 
        if result_forest > best_result_forest:
            best_model_forest = model_forest 
            best_result_forest = result_forest 
            best_est_forest = est
            best_depth_forest = depth
        print("max_depth =", depth, ": ", end='')
        print("n_estimators =", est, ": ", end='')
        print("Accuracy лучшей модели:", result_forest)
print("Accuracy наилучшей модели на валидационной выборке:", best_result_forest)
print('Количество деревьев:', best_est_forest)
print('Глубина дерева :', best_depth_forest)


max_depth = 1 : n_estimators = 1 : Accuracy лучшей модели: 0.7169517884914464
max_depth = 2 : n_estimators = 1 : Accuracy лучшей модели: 0.7387247278382582
max_depth = 3 : n_estimators = 1 : Accuracy лучшей модели: 0.7387247278382582
max_depth = 4 : n_estimators = 1 : Accuracy лучшей модели: 0.7589424572317263
max_depth = 5 : n_estimators = 1 : Accuracy лучшей модели: 0.7682737169517885
max_depth = 6 : n_estimators = 1 : Accuracy лучшей модели: 0.7651632970451011
max_depth = 7 : n_estimators = 1 : Accuracy лучшей модели: 0.7620528771384136
max_depth = 8 : n_estimators = 1 : Accuracy лучшей модели: 0.7636080870917574
max_depth = 9 : n_estimators = 1 : Accuracy лучшей модели: 0.7744945567651633
max_depth = 10 : n_estimators = 1 : Accuracy лучшей модели: 0.7822706065318819
max_depth = 11 : n_estimators = 1 : Accuracy лучшей модели: 0.776049766718507
max_depth = 12 : n_estimators = 1 : Accuracy лучшей модели: 0.76049766718507
max_depth = 13 : n_estimators = 1 : Accuracy лучшей модели: 0.74

Наилучшая модель на валидационной выборке при использовании случайный лес при accuracy = 0.806, глубина дерева depth = 10, кол-во деревьев 5


Модель логистическая регрессия

In [20]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=2000) 
model_log.fit(features_train, target_train) 
best_result_log = model_log.score(features_valid, target_valid) 

print("Accuracy наилучшей модели на валидационной выборке:", best_result_log)

Accuracy наилучшей модели на валидационной выборке: 0.7356143079315708


Значение Accuracy при модели логической регрессии составил 0.736

Таким образом, из трех моделей наилучший результат accuracy при использовании модели случайный лес, который составил 0.806, на втором месте модель дерево решений с результатом 0.773 и последнее место при логистической регрессии с результатом в 0.736

In [21]:
all_data = {'model': ['дерево решений','случвйный лес','логическая регрессия'],
        'programm':['DecisionTreeClassifier','RandomForestClassifier','LogisticRegression'],
        'accuracy':[0.773,0.806,0.736],
        'max_depth': [10,10,''],
        'n_estimators':['',5,'']
       }
all_results = pd.DataFrame(all_data)
  
display(all_results)       

,model,programm,accuracy,max_depth,n_estimators
0,дерево решений,DecisionTreeClassifier,0.77,10,
1,случвйный лес,RandomForestClassifier,0.81,10,5
2,логическая регрессия,LogisticRegression,0.74,,


## Проверьте модель на тестовой выборке

Проверим модель с наилучшим показателем accuracy на тестовой выборке

In [22]:
predictions_test =best_model_forest.predict(features_test) 
best_result_model_test=accuracy_score(target_test,predictions_test)
print("Значение accuracy на тестовой выборке:", best_result_model_test)

Значение accuracy на тестовой выборке: 0.7838258164852255


В соответсвии с заданием, Постройте модель с максимально большим значением accuracy ( не менее 0.75), были построены три модели: дерево решений, случайный лес и логистическая регрессия, из трех моделей наилучший результат показала модель случайного леса с показателем 0.806. две модели из трех превышают показатель 0.75. Использовав модель случайного леса на тестовой выборке, показатель accuracy составил 0.784, что также выше 0.75

## Проверьте модели на адекватность

In [23]:
adequacy = DummyClassifier(strategy='most_frequent', random_state=12345)
adequacy.fit(features_train, target_train)
adequacy_valid = adequacy.score(features_valid, target_valid)
adequacy_test = adequacy.score(features_test, target_test)

print('Значение  accuracy  валидационной выборки:', adequacy_valid)
print('Значение accuracy тестовой выборки:', adequacy_test)

Значение  accuracy  валидационной выборки: 0.6889580093312597
Значение accuracy тестовой выборки: 0.6951788491446346


При проверки на адекватность значения accuracy, как на валидационной выборке, так и на тестовой ниже, чем у выбранной  модели случайного леса, значит наша модель адекватна и эффективна.

Вывод:


1) изучили и предобработали предоставленные данные. В данных не содержалось дубликатов и пропусков. Для формирования модели, в рамках исключения мультипликарности,
исключили столбец с минутами, т.к. у него была большая корреляция с количеством звонков
2)  Данные были разбиты на три выборки: обучающую, валидационную и тестовую выборки в пропорции 3:1:1. 
3) Были построены три модели : дерево решений, случайный лес и логистическая регрессия.  наилучший показатель в 0.86 был у модели случайный лес,
    на втором месте оказалась модель дерево решений с показателем 0.773 и на третьем месте логистическая модель с показателем 0.736. 
    Таким образом, две модели из трех показали значение выше 0.75 заложенной в задании.
4) Налучшую модель проверили на тестовой выборке, получив показатель accuracy  0.784, что также лучше заложенного  в задании показателя 0.75
5) Проверили модель на адекватность, получив показатели accuracy, как на валидиционной, так и на тестовой выборке, ниже выбраного значения в модели случайного леса.

Таким образом, на основании ретраспективных данных компании и полученной модели,можно проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра»


